<img src="./images/03-faq.png" width=700px>


In this example we will see an example of RAG - Retrieval Augmented Generation.

We have 'retrieved' some data and stored in the FAQ variable. In this demo, this is hard coded but there are a multitude of ways of RETRIEVING data to AUGMENT the prompt and consequent GENERATION.

OpenAi does not have any domain specifice knowledge. In lieu of a fine tuned model on our domain, we can AUGMENT the prompt with retrieved data.

RAG is not just asking questions of documents using vector search - it is augmenting the LLM request with additonal data in lieu of fine tuning the model.

We are going to give it a list of FAQs so that we can use a ChatBot to ask questions.


In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

c:\Users\owner\Desktop\eval-framework\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_llm_client(llm_choice):
    if llm_choice == "GROQ":
        client = OpenAI(
            base_url="https://api.groq.com/openai/v1",
            api_key=os.environ.get("GROQ_API_KEY"),
        )
        return client
    elif llm_choice == "OPENAI":
        load_dotenv()  # load environment variables from .env fil
        client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
        return client
    else:
        raise ValueError("Invalid LLM choice. Please choose 'GROQ' or 'OPENAI'.")

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_AP#I_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

LLM_CHOICE = "OPENAI"
LLM_CHOICE = "GROQ"

if OPENAI_API_KEY:
    print(f"OPENAI_API_KEY exists and begins {OPENAI_API_KEY[:14]}...")
else:
    print("OPENAI_API_KEY not set")

if GROQ_API_KEY:
    print(f"GROQ_API_KEY exists and begins {GROQ_API_KEY[:14]}...")
else:
    print("GROQ_API_KEY not set")


client = get_llm_client(LLM_CHOICE)
if LLM_CHOICE == "GROQ":
    MODEL = "llama-3.3-70b-versatile"
else:
    MODEL = "gpt-4o-mini"

print(f"LLM_CHOICE: {LLM_CHOICE} - MODEL: {MODEL}")

OPENAI_API_KEY not set
GROQ_API_KEY exists and begins gsk_0yKDCuUXkz...
LLM_CHOICE: GROQ - MODEL: llama-3.3-70b-versatile


In [4]:
# A request to the LLM is stateless so we will always need to pass all the data that is needed each time.

# `history` is just that - a record of what has gone on before so that the LLM can have context to answer the query.


# We will use GRADIO as our UI.
def chat(message, history):
    # history is part of the gradio ChatInterface and it stores previous answers
    messages = (
        [{"role": "system", "content": system_message}]
        # + history ## groq adds metadata and causes error
        + [{"role": "user", "content": message}]
    )
    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)
    # ====================
    # AI bit
    stream = client.chat.completions.create(
        model=MODEL, messages=messages, stream=True, temperature=0.0
    )

    # Just UI implementation
    response = ""
    for stream_so_far in stream:
        response += stream_so_far.choices[0].delta.content or ""
        yield response

In [5]:
# We set up the backround scenario in the system messaage.
# In some frameworks this is called 'background' or 'character'.

system_message = (
    """You are a helpful assistant that provides answers to questions about PyData. """
)

In [6]:
pydata_southampton_facts = [
    "PyData Southampton is a local chapter of the global PyData community based in Southampton, UK.",
    "The group brings together data science enthusiasts, professionals, and academics from Southampton and the surrounding Hampshire area.",
    "PyData Southampton regularly hosts meetups featuring talks on data analysis, machine learning, and related Python technologies.",
    "The chapter has strong connections with the University of Southampton, particularly its data science and AI departments.",
    "PyData Southampton events typically include networking opportunities for local data professionals and students.",
    "The group welcomes members of all skill levels, from beginners to experienced data scientists and developers.",
    "PyData Southampton occasionally collaborates with other tech communities in the South of England for joint events.",
    "The chapter helps promote open-source Python tools and libraries for data analysis within the local tech ecosystem.",
    "PyData Southampton meetups often feature both technical presentations and more accessible introductory content.",
    "The group serves as an important hub for data science knowledge exchange in the Solent region of the UK.",
]

In [7]:
FAQ = [
    "Needle in a haystack? 42 communities in Southampton.", # Needle in a haystack?
    "PyData is a community for developers and users of data tools in Python, founded in 2012.",
    "The PyData ecosystem is maintained by NumFOCUS, a 501(c)(3) non-profit organization.",
    "PyData conferences bring together users and developers of data analysis tools to share ideas and learn from each other.",
    "The first PyData conference was held in 2012 in New York City.",
    "PyData events feature workshops, tutorials, and talks on data science, machine learning, and analytics.",
    "PyData has a global presence with conferences held across North America, Europe, Asia, Australia, and South America.",
    "Local PyData meetup groups exist in over 100 cities worldwide.",
    "PyData conferences typically run for 2-3 days, with additional workshop days sometimes offered.",
    "The PyData community emphasizes open-source software development.",
    "Key libraries in the PyData ecosystem include NumPy, pandas, Matplotlib, SciPy, and scikit-learn.",
    "PyData conferences have a code of conduct to ensure inclusivity and respect for all participants.",
    "Many PyData events offer financial aid programs to increase accessibility.",
    "PyData conferences often feature keynote speeches from prominent figures in data science and machine learning.",
    "The PyData YouTube channel - https://www.youtube.com/@PyDataTV - hosts recordings of talks from previous conferences.",
    "PyData events typically include networking opportunities like conference dinners or social events.",
    "The PyData community welcomes members of all skill levels, from beginners to experts.",
    "PyData conferences often have dedicated tracks for different skill levels or specialized topics.",
    "Many PyData conferences feature 'lightning talks' where participants can give short 5-minute presentations.",
    "PyData events sometimes include hackathons or sprints to contribute to open-source projects.",
    "Local PyData meetups often serve as a gateway for people to get involved with the broader community.",
    "Some popular annual PyData conferences include PyData London, PyData New York, and PyData Berlin.",
    "PyData conferences attract attendees from academia, industry, and government sectors.",
    "Speaker proposals for PyData conferences are typically selected through a community review process.",
    "Many PyData conferences are run primarily by volunteers from the local community.",
    "PyData often collaborates with other Python communities like SciPy and PyCon.",
    "The PyData website (pydata.org) serves as a central hub for conference information and community resources.",
    "PyData has developed into a trusted brand for high-quality technical content in the data science space.",
    "Many companies sponsor PyData events to connect with the data science community.",
    "PyData conferences often feature poster sessions where researchers can present their work.",
    "The PyData community emphasizes diversity and inclusion in both its membership and leadership.",
]

FAQ += pydata_southampton_facts
# Get total number of facts
print(f"Total number of facts: {len(FAQ)}")

Total number of facts: 41


In [8]:
# Create the base system message - 'character and instructions' along with retrieved data.
system_message += "\n" + "\n".join(FAQ) + "\n" + "\n".join(FAQ)

In [9]:
# We use Gradio for a chat interface
# prompt: I am interested in green belts, probably coming Sunday and will have cash. Is this OK?

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


History is:
[]
And messages is:
[{'role': 'system', 'content': "You are a helpful assistant that provides answers to questions about PyData. \nNeedle in a haystack? 42 communities in Southampton.\nPyData is a community for developers and users of data tools in Python, founded in 2012.\nThe PyData ecosystem is maintained by NumFOCUS, a 501(c)(3) non-profit organization.\nPyData conferences bring together users and developers of data analysis tools to share ideas and learn from each other.\nThe first PyData conference was held in 2012 in New York City.\nPyData events feature workshops, tutorials, and talks on data science, machine learning, and analytics.\nPyData has a global presence with conferences held across North America, Europe, Asia, Australia, and South America.\nLocal PyData meetup groups exist in over 100 cities worldwide.\nPyData conferences typically run for 2-3 days, with additional workshop days sometimes offered.\nThe PyData community emphasizes open-source software devel